In [2]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [48]:
def read_connection(db_password, db_user, db_host, db_port, db_name):
    engine= psycopg2.connect(
        user= db_user,
        password= db_password,
        host= db_host,
        port= db_port,
        database= db_name
        
    )
    return engine

def write_connection(user, password, host, port,db):
    engine= create_engine(
         f'postgresql://{user}:{password}@{host}:{port}/{db}'
    )
    engine.connect()
    return engine

try:
        db_engine= read_connection(db_password='postgres',db_user='postgres', db_host='localhost',db_port=5432, db_name='gold')
        print("Connection done successfully for reading...")
except Exception as e:
        print("Got ERROR in Connection to DB for Reading", e)

Connection done successfully for reading...


In [49]:
try:
        print("Test Connection for writing the Dimantions.")
        db_connection =write_connection('postgres', 'postgres','localhost', 5432,'gold')
        print("Connection Done (: ")
except Exception as e:
        print("Got ERROR in Connection for Writing...", e)

Test Connection for writing the Dimantions.
Connection Done (: 


In [6]:
try:    
        query = '''
            SELECT * 
            FROM retail_cleaned
        '''
        cleaned_data = pd.read_sql(query, db_engine, index_col='Id')
        print("The Cleaned Data Loaded Successfully")
except Exception as e:
        print("Got ERROR in your query or index", e)

C:\Users\Mohammed\AppData\Local\Temp\ipykernel_3588\3037261361.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cleaned_data = pd.read_sql(query, db_engine, index_col='Id')


The Cleaned Data Loaded Successfully


In [11]:

def Dimcustomer(cleaned_df, db_conn):
    DimCust = cleaned_df[['CustomerID', 'Country']].copy()
    DimCust['name'] = 'Customer' + DimCust['CustomerID'].astype(str)
    DimCust= DimCust.reset_index(drop=True)
    DimCust= DimCust.drop_duplicates('CustomerID').set_index('CustomerID')
    DimCust.head(n=0).to_sql(name='dq_dimcustomer', con=db_conn, if_exists='replace', index=True, index_label='CustomerID')


In [12]:
Dimcustomer(cleaned_data,db_connection)

In [50]:
query="""

    WITH dq_dimcustomer_columns AS (
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = 'dq_dimcustomer'
    ),

    dimcustomer_columns AS (
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = 'dimcustomer'
    )

    SELECT  *
    FROM dq_dimcustomer_columns AS dq_dimcustomer
    INNER JOIN dimcustomer_columns AS dimcustomer
    ON dq_dimcustomer.column_name = dimcustomer.column_name
    AND dq_dimcustomer.data_type = dimcustomer.data_type;

    """
df = pd.read_sql(query, db_engine)



C:\Users\Mohammed\AppData\Local\Temp\ipykernel_3588\2150218186.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, db_engine)


In [60]:
if len(df) == 3:
    print("Data Quality Check: All columns and data types match for the DimCustomer.")
    print(df)
else:
    print("Data Quality Check: Detected data quality issues in the DimCustomer.")


Data Quality Check: All columns and data types match for the DimCustomer.
  column_name data_type column_name data_type
0  CustomerID    bigint  CustomerID    bigint
1     Country      text     Country      text
2        name      text        name      text


In [53]:
df

<bound method DataFrame.count of   column_name data_type column_name data_type
0  CustomerID    bigint  CustomerID    bigint
1     Country      text     Country      text
2        name      text        name      text>

In [43]:
df= pd.read_csv(r'C:\Users\Mohammed\Desktop\retail-DWH\data\online_retail.csv')
df['InvoiceDate']= pd.to_datetime(df['InvoiceDate'])

In [44]:
df['CustomerID'] = df['CustomerID'].astype(int)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [37]:
df['MonthNo']= df['InvoiceDate'].dt.strftime('%m')
df['Day']= df['InvoiceDate'].dt.strftime('%d')


In [38]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,MonthNo,Day
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,12,01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,12,01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,12,01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,12,01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,12,01


In [39]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
MonthNo                object
Day                    object
dtype: object

In [35]:
df= df.drop_duplicates()

In [17]:
df.shape

(536641, 10)

In [27]:
df[['saleskey']].head()

,saleskey
0,0 17850.0\n1 17850.0\n2 ...
1,0 17850.0\n1 17850.0\n2 ...
2,0 17850.0\n1 17850.0\n2 ...
3,0 17850.0\n1 17850.0\n2 ...
4,0 17850.0\n1 17850.0\n2 ...


In [45]:

df['saleskey'] = df['CustomerID'].astype(str) + df['StockCode'] + df['MonthNo'] + df['Day']

df

KeyError: 'MonthNo'

In [ ]:
d